<a href="https://colab.research.google.com/github/vin136/llm-infer/blob/main/bench_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os

def get_csv_files_in_folders(base_folder="."):
    csv_files = []
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            # Check if the file has a .csv extension
            if file.lower().endswith(".csv"):
                # Construct the full path to the CSV file
                csv_files.append(os.path.join(root, file))

    return csv_files

In [4]:
files = get_csv_files_in_folders()

In [5]:
files

['./vllm/bench-vllm.csv',
 './mlc/bench-mlc-mistral.csv',
 './mlc/bench-mlc-llama2.csv']

In [6]:
dfs = [pd.read_csv(csv_file) for csv_file in files]
combined_df = pd.concat(dfs, ignore_index=True)

In [41]:
#let's compare
combined_df[['inference_engine','model','time']]

,inference_engine,model,time
0,vllm,mistralai/Mistral-7B-Instruct-v0.2,3.494984
1,vllm,mistralai/Mistral-7B-Instruct-v0.2,3.494304
2,vllm,mistralai/Mistral-7B-Instruct-v0.2,3.499816
3,vllm,mistralai/Mistral-7B-Instruct-v0.2,3.495971
4,vllm,mistralai/Mistral-7B-Instruct-v0.2,3.496787
5,vllm,mistralai/Mistral-7B-Instruct-v0.2,0.315968
6,vllm,mistralai/Mistral-7B-Instruct-v0.2,2.306152
7,vllm,mistralai/Mistral-7B-Instruct-v0.2,2.663163
8,mlc,Mistral-7B-Instruct-v0.2,1.580195
9,mlc,Mistral-7B-Instruct-v0.2,1.480922


In [18]:
#let's compare vllm vs mlc for mistral
vsel = (combined_df['inference_engine'] == 'vllm') & (combined_df['model'].str.startswith('mistral'))
df_vllm_mst = combined_df[vsel]


In [17]:
msel = (combined_df['inference_engine'] == 'mlc') & (combined_df['model'].str.startswith('Mistral'))
df_mlc_mst = combined_df[msel]


In [35]:
df_mlc_mst = df_mlc_mst.assign(word_cnt = lambda df: df['answer'].str.replace('\n','').str.split().str.len())


In [34]:
df_vllm_mst = df_vllm_mst.assign(word_cnt = lambda df: df['answer'].str.replace('\n','').str.split().str.len())


In [39]:
#time for generating 10-words
(df_vllm_mst['time'].sum()/df_vllm_mst['word_cnt'].sum())*10*1000

345.4802163957148

In [40]:
(df_mlc_mst['time'].sum()/df_mlc_mst['word_cnt'].sum())*10*1000

156.51549055199862